In [1]:
import sys
# MC: need to specify path to remo in notebook
# Windows
#local_path_to_repo =  'C:/Users/crows/Documents/GitHub/remo-python'
# Mac
local_path_to_repo = '/Users/melodi/Docs/remo-python'
sys.path.insert(0, local_path_to_repo)

In [2]:
%load_ext autoreload
%autoreload 2
import remo


    (\(\ 
    (>':') Remo server is running: {'version': '0.3.4'}
                


## Get Dataset

In [3]:
remo.list_datasets()

[Dataset 1 - 'Vova's dataset',
 Dataset 2 - 'open images detection',
 Dataset 3 - 'Subset of OID',
 Dataset 4 - 'train',
 Dataset 6 - 'test',
 Dataset 12 - 'oid 100',
 Dataset 13 - 'open images sample data',
 Dataset 19 - 'oid building ',
 Dataset 21 - 'oid building and car',
 Dataset 22 - 'oid building car detection',
 Dataset 25 - 'open images data',
 Dataset 26 - 'oid car',
 Dataset 28 - 'oid car and person',
 Dataset 29 - 'test upload']

In [3]:
my_dataset = remo.get_dataset(28)

In [5]:
my_dataset.view()

Open http://localhost:8000/datasets/28


![SegmentLocal](view_data.gif "segment")

In [5]:
my_dataset.view_annotation_statistics()

Open http://localhost:8000/annotation-detail/19/intro


![SegmentLocal](annotation.gif "segment")

## Train-Test Split

In [6]:
len(my_dataset)

200

In [13]:
val = my_dataset[0:len(my_dataset) // 3]

In [14]:
train = my_dataset[len(my_dataset) // 3:]

In [15]:
val.export_annotation_to_csv('val.csv')

In [16]:
train.export_annotation_to_csv('train.csv')

In [17]:
import pandas as pd
df = pd.read_csv('train.csv')

In [18]:
df.head()

,file_name,class,xmin,ymin,xmax,ymax,height,width
0,039b687dbee6ec4a.jpg,/m/01g317,103.863296,283.794432,226.743296,588.800256,768,1024
1,039b687dbee6ec4a.jpg,/m/01g317,383.268864,212.845824,558.811136,626.833920,768,1024
2,039b687dbee6ec4a.jpg,/m/01g317,542.720000,273.553920,664.868864,629.760000,768,1024
3,039b687dbee6ec4a.jpg,/m/01g317,683.154432,242.102784,811.154432,632.686080,768,1024
4,039b687dbee6ec4a.jpg,/m/01g317,733.623296,0.000000,1023.268864,767.268864,768,1024


## Prepare for Image Classification

### Arrange Train & Validation Data

In [19]:
import glob
import os
from shutil import copyfile
import pandas as pd

def arrange_folders(phase, cls, root='cls_dataset'):
    """
    Given dataframe and directory path containing the images
    Arranges samples as given below: 
        root/phase/class/xxx.ext
    Args:
        root: string. Root directory path.
        phase: string. 'train' or 'val'
        cls: string.
    """
    df = pd.read_csv(phase + '.csv')
    d = {'/m/01g317':'Person', '/m/0k4j':'Car'}
    df['class'] = df['class'].apply(lambda x: d[x])
    images = df.loc[df['class'] == cls].file_name.values
    phase_path = os.path.join(root, phase)
    if not os.path.exists(phase_path):
        os.mkdir(phase_path)
    class_path = os.path.join(phase_path, cls)
    if not os.path.exists(class_path):
        os.mkdir(class_path)
    for im in images:
        copyfile(os.path.join(root+cls, im), os.path.join(class_path, im))
        

In [22]:
arrange_folders(phase='val', cls='Car', root='/Users/melodi/remo-python/')

In [23]:
arrange_folders(phase='val', cls='Person', root='/Users/melodi/remo-python/')

In [24]:
arrange_folders(phase='train', cls='Car', root='/Users/melodi/remo-python/')

In [25]:
arrange_folders(phase='train', cls='Person', root='/Users/melodi/remo-python/')

Now we can continue to build our classification model with PyTorch.

## PyTorch

In [9]:
import torch
import torch.nn as nn
import torch.optim as optim
import numpy as np
import torchvision
from torchvision import datasets, models, transforms
import matplotlib.pyplot as plt
import copy

In [10]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

In [26]:
train_dataset = datasets.ImageFolder(os.path.join('/Users/melodi/remo-python/', 'train'), 
                                     transforms.Compose(
                                         [transforms.RandomResizedCrop(224),
                                          transforms.RandomHorizontalFlip(), 
                                          transforms.ToTensor(),
                                          transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])]))

In [65]:
train_dataloader = torch.utils.data.DataLoader(train_dataset, batch_size=4, shuffle=True, num_workers=4)

In [27]:
val_dataset = datasets.ImageFolder(os.path.join('/Users/melodi/remo-python/', 'val'), 
                                   transforms.Compose(
                                       [transforms.Resize(256),
                                        transforms.CenterCrop(224),
                                        transforms.ToTensor(),
                                        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])]))

In [28]:
val_dataloader =  torch.utils.data.DataLoader(val_dataset, batch_size=4, shuffle=True, num_workers=4)

In [29]:
datasizes = {x: len(val_dataset) if x == 'val' else len(train_dataset) for x in ['train','val'] }

In [30]:
datasizes 

{'train': 134, 'val': 66}

In [31]:
# load pre-trained resnet18
model_ft = models.resnet18(pretrained=True)
num_ftrs = model_ft.fc.in_features

num_classes = len(train_dataset.classes)
# reset the final fully connected layer
model_ft.fc = nn.Linear(num_ftrs, num_classes)


In [32]:
model_ft = model_ft.to(device)

criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(model_ft.parameters(), lr=0.001, momentum=0.9)


best_model_weights = copy.deepcopy(model_ft.state_dict())
best_acc = 0.0

In [72]:
epochs = 3

In [73]:
for epoch in range(epochs):      
    #print(f"Epoch {epoch}/{epochs-1}")
    #print(" ")
    for phase in ['train', 'val']:

        running_loss = 0.0
        running_corrects = 0

        if phase == 'train':
            model_ft.train()
            dataloader = train_dataloader
        else:
            model_ft.eval()
            dataloader = val_dataloader


        for images, labels in dataloader:
            images = images.to(device)
            labels = labels.to(device)

            optimizer.zero_grad()

            outputs = model_ft(images)
            _, preds = torch.max(outputs, 1)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()

            running_loss += loss.item() * images.size(0)
            running_corrects += torch.sum(preds == labels.data)
        else:
            epoch_acc = running_corrects.double() / datasizes[phase]
            epoch_loss = running_loss / datasizes[phase]
            #print(f"{phase} loss: {epoch_loss}")
            #print(f"{phase} acc: {epoch_acc}")

        if phase == 'val':
            if epoch_acc > best_acc:
                best_acc = epoch_acc
                best_model_weights = copy.deepcopy(model_ft.state_dict())

print(f'Best Val Acc: {best_acc}')
torch.save(model_ft.state_dict(), 'model.pth')
model_ft.load_state_dict(best_model_weights)

Best Val Acc: 0.9545454545454546


<All keys matched successfully>

In [35]:
model_ft.load_state_dict(torch.load('/Users/melodi/remo-python/example/model.pth'))

<All keys matched successfully>

In [36]:
class_names = train_dataset.classes

In [37]:
class_names

['Car', 'Person']

In [38]:
df_preds = pd.DataFrame(columns=['file_name','class'])
d = {'Person':'/m/01g317', 'Car':'/m/0k4j'}
with torch.no_grad():
    for i, (inputs, labels) in enumerate(val_dataloader,0):
        inputs = inputs.to(device)
        labels = labels.to(device)
        outputs = model_ft(inputs)
        _, preds = torch.max(outputs, 1)
        sample_fname, _ = val_dataloader.dataset.samples[i]
        sample_basename = sample_fname.split('/')[-1]
        for k in range(inputs.shape[0]):
            df_preds.loc[len(df_preds)+1] = [sample_basename, d[class_names[preds[k]]]]

In [39]:
df_preds.head()

,file_name,class
1,0004d295cfb46842.jpg,/m/0k4j
2,0004d295cfb46842.jpg,/m/0k4j
3,0004d295cfb46842.jpg,/m/0k4j
4,0004d295cfb46842.jpg,/m/0k4j
5,0007efdd465a14b8.jpg,/m/0k4j


In [40]:
df_preds.to_csv('preds_encoded.csv',index=False)